In [1]:
import pandas as pd
import numpy as np

In [2]:
data_folder = '/home/kate/Research/Property/Data/'

In [3]:
dataset = pd.read_csv(data_folder+'property_water_claims_non_cat_fs_v5.csv', error_bad_lines=False, index_col=False)

In [4]:
dataset['log_property_age']=np.where(dataset['cal_year']==dataset['yearbuilt'], 0, np.log(np.abs(dataset['cal_year'] - dataset['yearbuilt']))) 

/home/kate/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [5]:
dataset[['log_property_age','cal_year','yearbuilt']].sort_values(by=['log_property_age'])

,log_property_age,cal_year,yearbuilt
519525,0.000000,2016,2015
1037946,0.000000,2018,2019
229993,0.000000,2009,2009
1429856,0.000000,2019,2019
500410,0.000000,2016,2015
...,...,...,...
177762,4.795791,2021,1900
1232214,4.795791,2021,1900
126043,4.795791,2021,1900
1728973,4.795791,2021,1900


In [6]:
featureset  = [
'log_sqft',
'usagetype',    
'log_property_age',
'log_water_risk_3_blk',     
'cova_deductible',
'customer_cnt_active_policies',
'log_ecy',
'ecy'
]

In [7]:
target_column= 'cova_ic_nc_water' 

In [8]:
#StratifiedKFold
from sklearn.model_selection import StratifiedKFold
kfold = 5
skf = StratifiedKFold(n_splits=kfold, random_state=42, shuffle=True)

In [9]:
print('Creating training dataset excluding 2019 and 2020')
training_dataset=dataset[(dataset.cal_year != 2019) & (dataset.cal_year != 2020) ][featureset + [target_column]]
print('Training Size Original = %s'%len(training_dataset))
print('Creating folds')
for i, (train_index, test_index) in enumerate(skf.split(training_dataset[featureset], training_dataset[target_column])):
    print(' fold: {}  of  {} : '.format(i+1, kfold))
    training_dataset['fold_%s'%i]=0
    training_dataset.iloc[train_index,training_dataset.columns.get_loc('fold_%s'%i)]=1
print('Saving training dataset ')
training_dataset.to_csv('%sproperty_wcf_training.csv'%(data_folder),header=True,index=False)   

Creating training dataset excluding 2019 and 2020
Training Size Original = 1615387
Creating folds
 fold: 1  of  5 : 
 fold: 2  of  5 : 
 fold: 3  of  5 : 
 fold: 4  of  5 : 
 fold: 5  of  5 : 
Saving training dataset 


Training dataset for classification

In [10]:
testing_dataset=dataset[dataset.cal_year == 2019]
print('Testing Size = %s'%len(testing_dataset))
testing_dataset.to_csv('%sproperty_wcf_testing.csv'%(data_folder),header=True,index=False)   

Testing Size = 221557


In [ ]:
prediction_dataset=dataset[dataset.cal_year == 2020]
print('Prediction Size = %s'%len(prediction_dataset))
prediction_dataset.to_csv('%sproperty_wcf_prediction.csv'%(data_folder),header=True,index=False)